In [5]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import confusion_matrix
from data_loader import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import model
torch.set_printoptions(linewidth=120)
from utils import *
from collections import OrderedDict
from run_builder import RunBuilder
from run_manager import RunManager

from tqdm import tqdm
import time
import pandas as pd

from torch.utils.tensorboard import SummaryWriter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#device = 'cuda' if torch.cuda.is_available else 'cpu'
device = 'cpu'

In [3]:
params = OrderedDict(
    lr = [.01]
    ,batch_size = [10, 50]
    ,epochs = [3]
    ,num_workers = [0]
    ,batch_norm = [False, True]
    ,normalize = [False, True]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    comment = f'-{run}'
    
    data_loader = DataLoader('fashion_mnist',batch_size=run.batch_size, normalize=run.normalize)
    network = model.Network(run.batch_norm).to(device)
    optimizer = optim.Adam(
        network.parameters(), lr=run.lr
    )
    
    m.begin_run(run, network, data_loader)
    
    outer = tqdm(total=run.epochs, desc='Epoch', position=0)
    for epoch in range(run.epochs):
        m.begin_epoch()
        for batch in data_loader.train_loader:
            #inner = tqdm(total=batch[1].shape[0], desc='Batch', position=1)
            images = batch[0].to(device)
            labels = batch[1].to(device) # Get Batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
            #inner.update(1)
        outer.update(1)
        m.end_epoch()
    m.end_run()
m.save('results')
        



,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,epochs,num_workers,batch_norm,normalize
0,1,1,0.605651,0.777633,32.228973,32.443469,0.01,10,3,0,False,False
1,1,2,0.531774,0.813933,32.405049,64.965707,0.01,10,3,0,False,False
2,1,3,0.507931,0.823283,34.263286,99.339695,0.01,10,3,0,False,False
3,2,1,0.663035,0.758533,35.539749,35.616794,0.01,10,3,0,False,True
4,2,2,0.598022,0.793050,36.167524,71.893717,0.01,10,3,0,False,True
5,2,3,0.575913,0.798167,39.369509,111.376625,0.01,10,3,0,False,True
6,3,1,0.524714,0.809450,34.645520,34.734987,0.01,10,3,0,True,False
7,3,2,0.420848,0.850700,35.363302,70.243985,0.01,10,3,0,True,False
8,3,3,0.404563,0.856733,35.686324,106.074891,0.01,10,3,0,True,False
9,4,1,0.531445,0.805167,37.860121,37.945635,0.01,10,3,0,True,True


In [6]:
pd.DataFrame.from_dict(m.run_data).sort_values('accuracy',ascending=False)

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,epochs,num_workers,batch_norm,normalize
20,7,3,0.317050,0.883083,13.835417,41.276287,0.01,50,3,0,True,False
23,8,3,0.319929,0.881783,16.836489,51.509349,0.01,50,3,0,True,True
19,7,2,0.346792,0.872817,13.331882,27.307432,0.01,50,3,0,True,False
22,8,2,0.346549,0.872733,16.858071,34.531632,0.01,50,3,0,True,True
11,4,3,0.402921,0.858100,38.290665,114.901848,0.01,10,3,0,True,True
14,5,3,0.389409,0.856917,13.220616,41.198087,0.01,50,3,0,False,False
8,3,3,0.404563,0.856733,35.686324,106.074891,0.01,10,3,0,True,False
17,6,3,0.399120,0.854450,15.466557,47.961081,0.01,50,3,0,False,True
7,3,2,0.420848,0.850700,35.363302,70.243985,0.01,10,3,0,True,False
10,4,2,0.424027,0.849950,38.380503,76.473148,0.01,10,3,0,True,True
